In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.stats import norm
import itertools
import math
import scipy.stats


#gurobi key
options = {
    "WLSACCESSID": "de118b04-39b7-495b-b954-d47d9094c228",
    "WLSSECRET": "56fa884f-d64a-4bee-bf23-8d247f2990ec",
    "LICENSEID": 2581233,
}

## Mock data

In [ ]:
s_i= [0.71525707, 0.68319123, 0.8393927 , 0.76182247, 0.87573982,
       0.40073661, 0.82796312, 0.59724565, 0.38934696, 0.65373446,
       0.52556782, 0.64348453, 0.88315713, 0.63206879, 1.0777517 ,
       0.8088764 , 0.81647999, 0.88153565, 0.792893  , 0.78000662,
       0.7997215 , 0.28656563, 0.71634435, 1.05010094, 0.65473358,
       0.62464865, 0.66699049, 0.53609337, 0.8043781 ]
#road characteristic
alpha_i = [0.0525, 0.0725, 0.0725, 0.0725, 0.0725, 0.0525, 0.0525, 0.0525,
       0.0625, 0.0625, 0.0725, 0.0725, 0.0525, 0.0625, 0.0725, 0.0525,
       0.0625, 0.0525, 0.0725, 0.0725, 0.0625, 0.0725, 0.0725, 0.0725,
       0.0525, 0.0525, 0.0625, 0.0525, 0.0625]
beta_i= [0.007, 0.005, 0.003, 0.007, 0.003, 0.003, 0.007, 0.007, 0.003,
       0.005, 0.003, 0.005, 0.007, 0.003, 0.003, 0.005, 0.003, 0.003,
       0.007, 0.005, 0.005, 0.005, 0.003, 0.005, 0.005, 0.003, 0.005,
       0.007, 0.005]#np.random.choice([0.0035,0.0025], 30-1)

In [ ]:
# Initialize plot
plt.figure(figsize=(10, 6))

for i in range(29):
    mean = s_i[i] * alpha_i[i]
    variance = s_i[i] * beta_i[i] ** 2
    #print("i=", i, " mean=", mean, " variance=", variance)
    
    mu = mean
    sigma = math.sqrt(variance)
    
    # Create x values
    x = np.linspace(mu - 2.5*sigma, mu + 2.5*sigma, 100)*60
    plt.plot(x, scipy.stats.norm.pdf(x/60, mu, sigma) ,label=i)

# Customize the plot
plt.title('Normal Distribution PDFs')
plt.xlabel('Travel times between stations (min)')
plt.ylabel('Probability Density')
plt.legend()
plt.grid()
plt.show()  # Call show() after the loop to display all figures

In [ ]:
# data preparation
#set
M,N= 10,6
I=list(range(M))
J=list(range(N))
D=list(range(4))

#parameter
headway_scheduled=0.2
sigma_scheduled=0.005 ### check
alpha_instability=5

#segment distance
s_i= [0.71525707, 0.68319123, 0.8393927 , 0.76182247, 0.87573982,
       0.40073661, 0.82796312, 0.59724565, 0.38934696, 0.65373446,
       0.52556782, 0.64348453, 0.88315713, 0.63206879, 1.0777517 ,
       0.8088764 , 0.81647999, 0.88153565, 0.792893  , 0.78000662,
       0.7997215 , 0.28656563, 0.71634435, 1.05010094, 0.65473358,
       0.62464865, 0.66699049, 0.53609337, 0.8043781 ]
#road characteristic
alpha_i = [0.0525, 0.0725, 0.0725, 0.0725, 0.0725, 0.0525, 0.0525, 0.0525,
       0.0625, 0.0625, 0.0725, 0.0725, 0.0525, 0.0625, 0.0725, 0.0525,
       0.0625, 0.0525, 0.0725, 0.0725, 0.0625, 0.0725, 0.0725, 0.0725,
       0.0525, 0.0525, 0.0625, 0.0525, 0.0625]
beta_i= [0.007, 0.005, 0.003, 0.007, 0.003, 0.003, 0.007, 0.007, 0.003,
       0.005, 0.003, 0.005, 0.007, 0.003, 0.003, 0.005, 0.003, 0.003,
       0.007, 0.005, 0.005, 0.005, 0.003, 0.005, 0.005, 0.003, 0.005,
       0.007, 0.005]#np.random.choice([0.0035,0.0025], 30-1)

#demand stats
N_w=[26, 30, 20, 29, 20, 18, 38, 20, 29, 14, 0, 14, 27, 8, 19, 1, 30, 17, 30, 26, 19, 26, 2, 7, 22, 10, 25, 8, 4, 0, 32, 14, 0, 36, 29, 11, 19, 1, 11, 8, 9, 21, 0, 42, 18, 21, 9, 7, 2, 1, 15, 0, 26, 7, 17, 45, 32, 10, 29, 26, 29, 19, 8, 19, 18, 28, 20, 34, 15, 24, 9, 19, 23, 14, 33, 35, 36, 12, 18, 18, 16, 29, 20, 11, 20, 0, 8, 25, 33, 35, 25, 8, 7, 19, 53, 22, 34, 34, 22, 37, 31, 20, 8, 35, 0, 30, 38, 27, 11, 15, 2, 27, 9, 38, 19, 29, 26, 18, 3, 10, 32, 26, 10, 28, 12, 21, 55, 16, 23, 24, 15, 41, 5, 12, 21, 3, 27, 17, 5, 21]
N_w=N_w[:M]
capacity=50
N_f=[0 if i <capacity else i-capacity  for i in N_w]
N_w=[50 if i>capacity else i for i in N_w]

#control stop set, the largest demand stop
K=np.array(N_w).argsort().tolist()[::-1][:int(M/3)]
K.sort()

# driver type
mu_d=[1,1,1,1]
sigma_d=[1.2, 0.8,0.8,1.2]
CS_d=[0.7,1,0.7,1]

#driver type assignment count
f_d=[2,1,1,2]

## Optimization formulation

In [ ]:
# MIP formulation
with gp.Env(params=options) as env, gp.Model(env=env) as model:
    
    #variable 
    E_h=model.addVars(list(itertools.product(I, J)), name="E_h")
    VAR_h=model.addVars(list(itertools.product(I, J)), name="VAR_h")

    mu=model.addVars(J, name="mu")
    sigma=model.addVars(J ,lb=0, name="sigma")

    Mu=model.addVars(list(itertools.product(I, J )), lb=0, name="Mu")
    Sigma=model.addVars(list(itertools.product(I , J)), name="Sigma")

    pdf=model.addVars(list(itertools.product(I, J)),  name="pdf")
    cdf=model.addVars(list(itertools.product(I, J)),  name="cdf")

    E_w=model.addVars(list(itertools.product(I, J)), name="E_w")
    z=model.addVars(list(itertools.product(D, J)), vtype=GRB.BINARY, name="z")


    #auxiliary variable 
    Sigma_square=model.addVars(list(itertools.product(I, J)), lb=0, name="Sigma_square")
    Std=model.addVars(list(itertools.product(I, J)), name="sigma_square")
    pdf_by_1_n_cdf=model.addVars(list(itertools.product(I, J)), name="pdf_by_1_n_cdf")
    Std_pdf_by_1_n_cdf=model.addVars(list(itertools.product(I, J)), name="Std_pdf_by_1_n_cdf")
    pdf_by_1_n_cdf_square=model.addVars(list(itertools.product(I, J)),lb=0,  name="pdf_by_1_n_cdf_square")

    E_h_square=model.addVars(list(itertools.product(I, J)),lb=0, name="E_h_square")
    COV=model.addVars(list(itertools.product(I, J)), name="E_h_square")


    

    #constraints 
    for i in I:
        model.addConstr( E_h[i, 0] == headway_scheduled, name='E_h' + "(" + str(i) +",0 )")
        model.addConstr( VAR_h[i, 0] == sigma_scheduled**2, name='VAR_h' + "(" + str(i) +",0 )")
       # model.addConstr( Mu[i, 0] == headway_scheduled, name='Mu' + "(" + str(i) +",0 )")
        #model.addConstr( Sigma[i, 0] == sigma_scheduled**2, name='Sigma' + "(" + str(i) +",0 )")

    for j  in J[1:]:
        model.addConstr( E_h[0, j] == headway_scheduled, name='E_h' + "(0," +str(i) +" )")
        model.addConstr( VAR_h[0, j] == sigma_scheduled**2, name='VAR_h' +"(0," +str(i) +" )")
        #model.addConstr( Mu[0, j] == headway_scheduled, name='Mu' + "(0," +str(i) +" )")
        #model.addConstr( Sigma[0, j] == sigma_scheduled**2, name='Sigma' +"(0," +str(i) +" )")
        

    #Var(h_ij) and E(h_ij)
    for i in I[1:]:
        for j in J[1:]:
            model.addConstr(E_h[i,j]==Mu[i,j]+Sigma[i,j]* pdf_by_1_n_cdf[i,j]
                            , name='E_h' + "(" + str(i) + "," +str(j)+")")
            model.addConstr(VAR_h[i,j]== Sigma_square[i,j]*(1-Std_pdf_by_1_n_cdf[i,j]-pdf_by_1_n_cdf_square[i,j] ),
                              name='VAR_h' + "(" + str(i) + "," +str(j)+")")

    # Mu, Sigma
    for i in I[1:]:
        for j in J[1:]:
            model.addConstr( Mu[i,j]==E_h[i-1,j]+E_w[i-1,j]-E_w[i-1,j-1]+alpha_i[i-1]*s_i[i-1]*(mu[j]- mu[j-1]) 
                           , name='Mu' + "(" + str(i) + "," +str(j)+")")
           
            model.addConstr( Sigma_square[i,j]== beta_i[i-1]**2 *s_i[i-1]*(sigma[j]**2 +sigma[j-1]**2)
                           , name='sigma_square' + "(" + str(i) + "," +str(j)+")")
            
    #mu, sigma
    for j in J:
        model.addConstr( mu[j] == gp.quicksum(z[d,j]* mu_d[d] for d in D), name='mu' + "(" + str(j) + ")")
        model.addConstr( sigma[j] == gp.quicksum(z[d,j]* sigma_d[d] for d in D), name='sigma' + "(" + str(j) + ")") 

    #E_w
    for i in  K:
        for j in J[1:]:
             model.addConstr( E_w[i,j]==gp.quicksum(z[d,j]* CS_d[d]*(headway_scheduled- E_h[i,j]+ E_w[i,j-1])   for d in D)
                           , name='E_w' + "(" + str(i) + "," +str(j)+")")
    
    for i in I:
        if i not in K:
            model.addConstr( E_w[i,0]==0 , name='E_w' + "(" + str(i) + "," +str(0)+")")
    
    #z[d,j]
    for j in J:
        model.addConstr( gp.quicksum(z[d,j] for d in D)==1 , name='z' + "("  +str(j)+")")
    
    for d in D:
        model.addConstr( gp.quicksum(z[d,j] for j in J)==f_d[d] , name='z_supply' + "("  +str(d)+")")

    #fixed z_d_j
    para_z=np.zeros((len(D), len(J)))
    para_z[0,:2]=1
    para_z[1,2]=1
    para_z[2,3]=1
    para_z[3,4:]=1
    for d in D:
        for j in J:
            model.addConstr(z[d,j]==int(para_z[d,j] ), name='fixed z' + "(" + str(d)+","+str(j) + ")")

    #pdf, cdf        
    for i in I:
        for j in J:
            model.addGenConstrPWL(Std[i,j], cdf[i,j], [-10.0, -2.04, -0.96, 0.96, 2.04, 10.0],
                                   [-0.002, 0.004, 0.148, 0.852, 0.996, 1.002])
            model.addGenConstrPWL(Std[i,j], pdf[i,j],[-10.0, -2.71, -1.78, -0.3, 0.3, 1.78, 2.72, 10.0]
                                  ,[-0.001, 0.002, 0.072, 0.393, 0.393, 0.071, 0.002, 0] )

    

    #auciliary constraints
    for i in I:
        for j in J:
            model.addConstr(Mu[i,j]==Sigma[i,j]*Std[i,j] , name='Std' + "(" + str(i)+","+str(j) + ")")
            model.addConstr(pdf_by_1_n_cdf[i,j]*(1-cdf[i,j])==pdf[i,j] , name='pdf_cdf' + "(" + str(i)+","+str(j) + ")")
            model.addConstr(pdf_by_1_n_cdf[i,j]*pdf_by_1_n_cdf[i,j]==pdf_by_1_n_cdf_square[i,j] 
                            , name='pdf_by_1_n_cdf_square' + "(" + str(i)+","+str(j) + ")")
            model.addConstr(Std[i,j]*pdf_by_1_n_cdf[i,j]==Std_pdf_by_1_n_cdf[i,j] 
                            , name='Std_pdf_by_1_n_cdf' + "(" + str(i)+","+str(j) + ")")
            

    for i in I:
        for j in J:
            model.addConstr(E_h_square[i,j]==E_h[i,j] *E_h[i,j] 
                            , name='E_h_square' + "(" + str(i)+","+str(j) + ")")
            model.addConstr(COV[i,j]*E_h_square[i,j]==VAR_h[i,j]
                            , name='COV' + "(" + str(i)+","+str(j) + ")")
            
    

        

    #objective function
    obj = model.setObjective(gp.quicksum(0.5*E_h[i,j]*(1+COV[i,j])*N_w[i] +E_h[i,j]*N_f[i] for i in I for j in J)
                             + alpha_instability* gp.quicksum(COV[i,j] for j in J)
                             , GRB.MINIMIZE)

    #solve
    model.optimize()
    if model.status != GRB.Status.OPTIMAL:
        print("***** Gurobi solve status:", model.status)
        print("***** This problem instance does not have an optimal solution")
    else:  # some fancy output formatting; we'll learn what the different components of the output are
        print(" ")
        print("primal var")



In [ ]:
para_z=np.zeros((len(D), len(J)))
para_z[0,:2]=1
para_z[1,2]=1
para_z[2,3:5]=1
para_z[3,5]=1

## process constraints

## convert normal distribution CDF to general constraints

### Summation of small inteval

### Polynomial function

In [ ]:
import numpy as np
from pylab import *

# Create some test data
dx = 0.01
X  = np.arange(-2, 2, dx)
Y  = np.exp(-X ** 2)

# Normalize the data to a proper PDF
Y /= (dx * Y).sum()

# Compute the CDF
CY = np.cumsum(Y * dx)

# Plot both
plot(X, Y)
plot(X, CY, 'r--')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from scipy.stats import norm

# Creating x-coordinates
x=np.linspace(-10,10,10000)
# Degree 2 equation
y=norm.cdf(x)

plt.scatter(x, y)
plt.xlabel("X-values")
plt.ylabel("Y-values")


# Using polyfit() to generate coefficients
error=[]
y_pred={}
p={}
for degree in range(20):
    p[degree] = np.polyfit(x, y, degree)  # Last argument is degree of polynomial
    
    
    
    # poly1d helps to define a polynomial function
    predict = np.poly1d(p[degree])
    # Test value
    y_pred[degree] = predict(x)
    
    score= sum([(y_pred[degree][i]-y[i] )**2  for i in range(len(x))])
    error.append(score)

choice_degree=4
plt.plot(x, y_pred[choice_degree],"r")
print("Selected degree=",choice_degree,"\n", "Coeficient values:", p[choice_degree])

In [ ]:
print(norm.cdf(-2.5)*100,norm.cdf(2.5)*100)

In [ ]:
plt.plot(list(range(20)), error,"r")
plt.title(" error by degree")

### piecewise-linear function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pwlf


x=np.linspace(-10,10,1000)
y=norm.cdf(x)

degrees=list(range(3,10))
my_pwlf={}
y_hat={}
score={}

for degree in degrees:
    my_pwlf[degree] = pwlf.PiecewiseLinFit(x, y)
    breaks = my_pwlf[degree].fit(degree)
    print(breaks)

    y_hat[degree] = my_pwlf[degree].predict(x)
    score[degree]= sum([(y_hat[degree][i]-y[i] )**2  for i in range(len(x))])


plt.figure()
# plt.plot(x, y, 'o')
# plt.plot(x_hat, y_hat, '-')
plt.plot(degrees, score, '-')
plt.show()

In [ ]:
plt.figure()
# plt.plot(x, y, 'o')
# plt.plot(x_hat, y_hat, '-')
plt.plot(score.keys(),score.values(), '-')
plt.show()

In [ ]:
plt.figure()
plt.plot(x, y, 'o',markersize=3)
plt.plot(x, y_hat[5], '-',markersize=0.5, alpha=0.8)
plt.show()

In [ ]:
from sympy import Symbol
from sympy.utilities import lambdify
x = Symbol('x')


def get_symbolic_eqn(pwlf_, segment_number):
    if pwlf_.degree < 1:
        raise ValueError('Degree must be at least 1')
    if segment_number < 1 or segment_number > pwlf_.n_segments:
        raise ValueError('segment_number not possible')
    # assemble degree = 1 first
    for line in range(segment_number):
        if line == 0:
            my_eqn = pwlf_.beta[0] + (pwlf_.beta[1])*(x-pwlf_.fit_breaks[0])
        else:
            my_eqn += (pwlf_.beta[line+1])*(x-pwlf_.fit_breaks[line])
    # assemble all other degrees
    if pwlf_.degree > 1:
        for k in range(2, pwlf_.degree + 1):
            for line in range(segment_number):
                beta_index = pwlf_.n_segments*(k-1) + line + 1
                my_eqn += (pwlf_.beta[beta_index])*(x-pwlf_.fit_breaks[line])**k
    return my_eqn.simplify()


choice_degree=5
eqn_list = []
f_list = []
for i in range(my_pwlf[choice_degree].n_segments):
    eqn_list.append(get_symbolic_eqn(my_pwlf[choice_degree], i + 1))
    print('Equation number: ', i + 1)
    print(eqn_list[-1])
    f_list.append(lambdify(x, eqn_list[-1]))

print("break point:","\n", my_pwlf[choice_degree].fit(choice_degree))


In [ ]:
my_pwlf[choice_degree].fit(choice_degree)

In [ ]:

my_pwlf[degree].predict(my_pwlf[choice_degree].fit(choice_degree))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pwlf


x=np.linspace(-2.5,2.5,100)
y_pdf=norm.pdf(x)

degrees=list(range(3,10))
pdf_pwlf={}
y_pdf_hat={}
breaks_pdf={}
score_pdf={}

for degree in degrees:
    pdf_pwlf[degree] = pwlf.PiecewiseLinFit(x, y_pdf)
    breaks_pdf[degree] = pdf_pwlf[degree].fit(degree)

    y_pdf_hat[degree] = pdf_pwlf[degree].predict(x)
    score_pdf[degree]= sum([(y_pdf_hat[degree][i]-y_pdf[i] )**2  for i in range(len(x))])


plt.figure()
plt.plot(score_pdf.keys(), score_pdf.values(), '-')
plt.show()

In [ ]:
choice_degree=6
plt.figure()
plt.plot(x, y_pdf, 'o',markersize=2)
plt.plot(x, y_pdf_hat[choice_degree], '-',markersize=1.5)
plt.show()

In [ ]:
breaks_pdf[choice_degree]

In [ ]:
pdf_pwlf[degree].predict(breaks_pdf[choice_degree].fit(choice_degree))

In [ ]:
# truncated normal distribution without the variance
import numpy as np
import matplotlib.pyplot as plt
import pwlf


def trunc_variance_1(x,y):
    return y**2*(1- (x/y)*norm.pdf(-x/y)/(1-norm.cdf(-x/y)) -  (norm.pdf(-x/y)/(1-norm.cdf(-x/y)))**2 )

def trunc_mean_1(x,y):
    return x+y*norm.pdf(-x/y)/(1-norm.cdf(-x/y))

def back_sigma(x,y):
    return y**2


mu=np.linspace(0, 10,1000)
sigma=np.linspace(0, 10,1000)
x_mu, y_sigma= np.meshgrid(mu, sigma)
z_manual =np.array(trunc_variance_1(np.ravel(x_mu), np.ravel(y_sigma) ) )
z_scipy = np.array(scipy.stats.truncnorm.var( 0, np.inf, loc=np.ravel(x_mu), scale=np.ravel(y_sigma) ))
z_normal=np.array(back_sigma(np.ravel(x_mu), np.ravel(y_sigma) ) )

Z_manual = z_manual.reshape(x_mu.shape)
Z_scipy = z_scipy.reshape(x_mu.shape)
Z_normal= z_normal.reshape(x_mu.shape)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(x_mu, y_sigma, Z_manual,label="Z_manual")
ax.plot_surface(x_mu, y_sigma, Z_normal ,label="Z_normal")
ax.plot_surface(x_mu, y_sigma, Z_scipy,label= "Z_scipy")

ax.set_xlabel('mu')
ax.set_ylabel('sigma')
ax.set_zlabel('variance')

In [ ]:
scipy.stats.truncnorm.var( 0,100, loc=4, scale=2 )

In [ ]:
scipy.stats.norm.var( loc=4, scale=2)

In [ ]:
mu=np.linspace(0, 10,1000)
sigma=np.linspace(0, 10,1000)
x_mu, y_sigma= np.meshgrid(mu, sigma)
z_manual_mean =np.array(trunc_mean_1(np.ravel(x_mu), np.ravel(y_sigma) ) )
z_scipy_mean = np.array(scipy.stats.truncnorm.mean( 0, np.inf, loc=np.ravel(x_mu), scale=np.ravel(y_sigma) ))

Z_manual_mean = z_manual_mean.reshape(x_mu.shape)
Z_scipy_mean = z_scipy_mean.reshape(x_mu.shape)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(x_mu, y_sigma, Z_manual_mean)
ax.plot_surface(x_mu, y_sigma, Z_scipy_mean)

ax.set_xlabel('mu')
ax.set_ylabel('Sigma')
ax.set_zlabel('Mean manual')


In [ ]:
print(trunc_variance_1(12,4))
print(scipy.stats.truncnorm.var( 0, np.inf,12,4))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(x_mu, y_sigma, Z_scipy)

ax.set_xlabel('mu')
ax.set_ylabel('sigma')
ax.set_zlabel('Variance of Truncated from scipy')

In [ ]:
scipy.stats.truncnorm.var( 0, np.inf, loc=1 ,scale=5 )

In [ ]:
x_test=np.linspace(0,10, 100)
y_test=1
plt.plot(x_test,trunc_variance_1(x_test,y_test))

In [ ]:
trunc_variance_1(4,1)

In [ ]:
trunc_variance_1(4,y_test)

In [ ]:
plt.plot(x_test,scipy.stats.truncnorm.var( 0, np.inf, loc=x_test, scale=y_test  )) 


In [ ]:
res_list=[]
for a_lower_bound in range(10):

    trunc_rand_sample=list(scipy.stats.truncnorm.rvs(a_lower_bound, np.inf, loc=10, scale=1 , size=1000000 ))
    res=np.var(trunc_rand_sample)
    res_list.append(res)

plt.plot(list(range(10)), res_list)


In [ ]:

scipy.stats.truncnorm.mean(a=0, b=np.inf, loc=1, scale=4  )

In [ ]:
res=scipy.stats.truncnorm.rvs(a=0, b=np.inf, loc=1, scale=2 , size=1000 )